# Panel Demo

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

This notebook demonstrates the use of **Holoviz [Panel](https://panel.holoviz.org/index.html)**. 

*Panel is an open-source Python library that lets you create custom interactive web apps and dashboards by connecting user-defined widgets to plots, images, tables, or text.*

In [ ]:
import urllib
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from pyproj import CRS
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

import panel as pn
import panel.widgets as pnw

In [ ]:
pn.extension(sizing_mode="stretch_width")

In [ ]:
try: 
    df = read_file('../data/geolife_small.gpkg')
except:
    df = read_file(os.path.join(os.getcwd(),'data/geolife_small.gpkg'))

df['t'] = pd.to_datetime(df['t'])
df = df.set_index('t').tz_localize(None)
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id')

In [ ]:
generalizers=['DouglasPeuckerGeneralizer', 'MinDistanceGeneralizer', 'TopDownTimeRatioGeneralizer']

In [ ]:
def plot_generalized(traj_id=1, tolerance=10, generalizer='DouglasPeuckerGeneralizer'):
    my_traj = traj_collection.get_trajectory(traj_id).to_crs(CRS(4088))
    if generalizer=='DouglasPeuckerGeneralizer':
        generalized = mpd.DouglasPeuckerGeneralizer(my_traj).generalize(tolerance)
    elif generalizer=='TopDownTimeRatioGeneralizer':
        generalized = mpd.TopDownTimeRatioGeneralizer(my_traj).generalize(tolerance)
    else:
        generalized = mpd.MinDistanceGeneralizer(my_traj).generalize(tolerance)
    generalized.add_speed(overwrite=True)
    reduction_pct = 1-generalized.size()/my_traj.size()
    return ( 
        generalized.hvplot(title=f'Trajectory {my_traj.id} (tolerance={tolerance}, {round(reduction_pct*100,1)}% reduction)', c='speed', cmap='Viridis', colorbar=True, clim=(0,20), line_width=10, width=500, height=500) + 
        generalized.df['speed'].hvplot.hist(title='Speed histogram', width=300, height=500) 
        )

In [ ]:
kw = dict(traj_id=(1, len(traj_collection)), tolerance=(0, 100, 10), generalizer=generalizers)
pn.interact(plot_generalized, **kw)

## App

Lets wrap this interactive panel into a nice template that can be served via 
`panel serve .\panel.ipynb --show --autoreload`

In [ ]:
from panel.template import DarkTheme

pn.template.FastListTemplate(theme=DarkTheme, 
    site="Panel", title="MovingPandas Generalizer Demo", 
    main=[pn.interact(plot_generalized, **kw)]
).servable();